In [4]:


import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import pickle
import string


with open("../../datasets/cleaned_balanced.txt", encoding='utf-8') as f:
    text = f.read()

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 5  # Number of input words

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i-seq_length:i+1])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length+1, padding='pre'))

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

# Build LSTM Model
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=seq_length))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#  Train the model
model.fit(X, y, epochs=30, batch_size=128)

#  Save model and tokenizer
model.save("next_word_model.h5")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
# Predict the next word
def predict_next_word(seed_text):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)[0]

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            return word
    return ""


C:\Users\admin\anaconda3\envs\tenserflow_env\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)             │ ?                        │   0 (unbuilt) │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ lstm (LSTM)                       │ ?                        │   0 (unbuilt) │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ dense (Dense)                     │ ?                        │   0 (unbuilt) │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 292s 159ms/step - accuracy: 0.0975 - loss: 7.2183
Epoch 2/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 274s 151ms/step - accuracy: 0.1487 - loss: 6.1614
Epoch 3/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 283s 156ms/step - accuracy: 0.1731 - loss: 5.7487
Epoch 4/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 279s 154ms/step - accuracy: 0.1887 - loss: 5.4132
Epoch 5/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 314s 150ms/step - accuracy: 0.2066 - loss: 5.0835
Epoch 6/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 272s 150ms/step - accuracy: 0.2205 - loss: 4.7923
Epoch 7/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 270s 149ms/step - accuracy: 0.2414 - loss: 4.5034
Epoch 8/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 252s 139ms/step - accuracy: 0.2680 - loss: 4.2138
Epoch 9/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 293s 162ms/step - accuracy: 0.2932 - loss: 3.9723
Epoch 10/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 259s 143ms/step - accuracy: 0.3212 - loss: 3.7340
Epoch 11/30
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 267s 146ms/step - accuracy: 0.3523 -

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step
Next word prediction: brave


In [10]:
def generate_text(seed_text, next_words=5):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Example:
print(generate_text("shivaji maharaj was", next_words=10))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
shivaji maharaj was the title of sirnobut and commander the cavalry also put
